In [2]:
from diffusers import StableDiffusionPipeline
import torch

# 모델 이름
model_name = "stabilityai/sdxl-turbo"

try:
    # 모델 로드
    pipeline = StableDiffusionPipeline.from_pretrained(
        model_name, 
        torch_dtype=torch.float16,  # GPU 사용 시 float16을 권장
        use_auth_token=True  # CLI를 통해 로그인한 경우 자동으로 토큰 사용
    )
    print("모델이 성공적으로 로드되었습니다.")
except Exception as e:
    print(f"모델 로드 중 오류 발생: {e}")


c:\Users\ParkChunSoo\anaconda3\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ParkChunSoo\.cache\huggingface\hub\models--stabilityai--sdxl-turbo. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Fetching 18 files: 100%|██████████| 18/18 [03:58<00:00, 13.26s/it]
Keyword arguments {'use_auth_token': T

모델이 성공적으로 로드되었습니다.
